In [1]:
%pip install -qU langchain-ollama
%pip install -qU langchain

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import langchain
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

## Model

In [3]:
chat = ChatOllama(
    model = "llama3.2",
    temperature=0,
)

In [4]:
chat

ChatOllama(model='llama3.2', temperature=0.0)

### Prompt template

In [5]:
template_string = """Translate the text\
that is delimited by triple backticks\
into a style that is {style}. \
text:```{text}```
"""

In [6]:
prompt_template = ChatPromptTemplate.from_template(template_string)

In [7]:
prompt_template.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the textthat is delimited by triple backticksinto a style that is {style}. text:```{text}```\n'), additional_kwargs={})]

In [8]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the textthat is delimited by triple backticksinto a style that is {style}. text:```{text}```\n')

In [9]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [10]:
customer_style = '''American English\
in a calm and respectful tone
'''

In [11]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [12]:
customer_messages = prompt_template.format_messages(
    style=customer_style,
    text=customer_email
)

In [13]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [14]:
print(customer_messages[0])

content="Translate the textthat is delimited by triple backticksinto a style that is American Englishin a calm and respectful tone\n. text:```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


In [15]:
customer_response = chat(customer_messages)

/tmp/ipykernel_35421/3256373024.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  customer_response = chat(customer_messages)


In [16]:
print(customer_response)

content='Here\'s a translation of the text into American English in a calm and respectful tone:\n\n"I\'m really frustrated that my blender lid came loose and spilled smoothie all over my kitchen walls. To make matters worse, the warranty doesn\'t cover the cost of cleaning up my kitchen. I could use some assistance right now."\n\nI made the following changes to adapt the text to American English:\n\n- Changed "Arrr" to a more neutral greeting\n- Replaced "me" with "my"\n- Changed "fuming" to "frustrated"\n- Replaced "matey" with a more polite phrase ("I could use some assistance")\n- Made minor adjustments to sentence structure and wording for clarity and flow.' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2024-10-06T14:08:21.020600377Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3656005554, 'load_duration': 1462651373, 'prompt_eval_count': 108, 'prompt_eval_duration': 51170000, 'eval_count': 1

In [17]:
print(customer_response.content)

Here's a translation of the text into American English in a calm and respectful tone:

"I'm really frustrated that my blender lid came loose and spilled smoothie all over my kitchen walls. To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could use some assistance right now."

I made the following changes to adapt the text to American English:

- Changed "Arrr" to a more neutral greeting
- Replaced "me" with "my"
- Changed "fuming" to "frustrated"
- Replaced "matey" with a more polite phrase ("I could use some assistance")
- Made minor adjustments to sentence structure and wording for clarity and flow.


In [18]:
service_reply = """Hey there customer, \
the warranty dose not cover\
cleaning expenses for your kitchen \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [19]:
service_style_pirate = """
a polit tone \
that speaks in English Pirate \
"""

In [20]:
service_message = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply
)

In [21]:
print(service_message[0].content)

Translate the textthat is delimited by triple backticksinto a style that is 
a polit tone that speaks in English Pirate . text:```Hey there customer, the warranty dose not covercleaning expenses for your kitchen you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [22]:
service_response = chat(service_message)
print(service_response.content)

Arrrr, ye landlubber!

Let's rewrite that scurvy message in a more... shall we say, "pirate-friendly" tone:

`Ahoy, matey! 

We've received yer complaint about the warranty not coverin' them cleanin' expenses fer yer kitchen blender. Now, we know ye might be thinkin', "But I put the lid on, savvy?" Alas, it seems like ye forgot to do so before startin' up that contraption.

Don't ye worry, matey! We'll make sure to pass along yer concerns to our trusty quartermaster (that's our customer service team, fer landlubbers). They'll be happy to help ye navigate the waters o' warranty coverage and get ye back on track with yer kitchen adventures!

Fair winds and following seas!`

(Note: I've kept the pirate slang and tone, but made sure it's still clear and respectful. No need to ruffle any feathers!)


## Output Parsers


In [23]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [24]:
prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [25]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOllama(model="llama3.2",
                temperature=0)
response = chat(messages)
print(response.content)

Here is the extracted information in JSON format:

```
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there", "It arrived in two days, just in time for my wife's anniversary present."]
}
```


In [26]:
type(response.content)

str

In [27]:
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

### Parse the LLM output string into a Python dictionary

In [28]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [29]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [32]:
print(output_parser)

response_schemas=[ResponseSchema(name='gift', description='Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.', type='string'), ResponseSchema(name='delivery_days', description='How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.', type='string'), ResponseSchema(name='price_value', description='Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.', type='string')]


In [30]:
format_instructions = output_parser.get_format_instructions()

In [31]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [33]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [34]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```

In [35]:
response = chat(messages)

In [36]:
print(response.content)

```json
{
    "gift": "False",
    "delivery_days": "2",
    "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [37]:
output_dict = output_parser.parse(response.content)

In [38]:
output_dict

{'gift': 'False',
 'delivery_days': '2',
 'price_value': "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}

In [39]:
type(output_dict)

dict

In [40]:
output_dict.get('delivery_days')

'2'